In [78]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
import re

In [14]:
link = "https://www.arup.com/perspectives/cities"
driver = webdriver.Firefox()
driver.get(link)
time.sleep(5)
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
while True:
    driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
    try:
        button = driver.find_element(By.CSS_SELECTOR, ".cta.cta--black.cta--plus.cta--plain.load-more")
        button.click()  
        time.sleep(1)
    except NoSuchElementException:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [31]:
headers = [header.text for header in soup.select('h3.preview-copy__title')]
urls = ['https://www.arup.com' + link['href'] for link in soup.select('a.preview-copy__toplink')]

In [32]:
links = pd.DataFrame(columns=['title', 'url'])
links.title = headers
links.url = urls

In [34]:
links

,title,url
0,"People, nature and technology: an integrated v...",https://www.arup.com/perspectives/people-natur...
1,"Planting an idea: Tirana's radical, orbital gr...",https://www.arup.com/perspectives/planting-an-...
2,Relighting and rewilding our environment,https://www.arup.com/perspectives/relighting-a...
3,What if the client was everyone? How social va...,https://www.arup.com/perspectives/what-if-the-...
4,"People, nature and technology: an integrated v...",https://www.arup.com/perspectives/people-natur...
...,...,...
269,BiodiverCities by 2030: transforming cities re...,https://www.arup.com/perspectives/publications...
270,Deadline 2020: How cities will get the job done,https://www.arup.com/perspectives/publications...
271,Pushing the boundaries of BIM,https://www.arup.com/perspectives/pushing-the-...
272,How charrettes help city planning,https://www.arup.com/perspectives/how-charrett...


In [80]:
corpus = pd.DataFrame(columns=['title', 'url', 'date', 'text'])

In [81]:
links.loc[0]

title    People, nature and technology: an integrated v...
url      https://www.arup.com/perspectives/people-natur...
Name: 0, dtype: object

In [85]:
for i, link in tqdm(links.iterrows(), total=len(links)):
    try:
        url = link.url
        title = link.title
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        article = soup.article.text
        text = re.sub(r'[\r\n\xa0 ]+', ' ', article).strip()
        corpus.loc[i] = [title, url, np.nan, text]
    except Exception as e:
        corpus.loc[i] = np.nan
        print(f'Error at index {i}: {e}')

  0%|          | 0/274 [00:00<?, ?it/s]

Error at index 96: 'NoneType' object has no attribute 'text'


In [90]:
corpus.loc[4, 'text']

"A net zero built environment, given the sector’s energy intensive construction processes and reliance on emissions-producing materials like concrete and steel and glass, is rightly recognised as an incredibly tough challenge. Across the world practitioners are evaluating how this global goal might be locally realised. In high rise, high density, sub-tropical cities like Hong Kong it’s already clear that we will need to use a highly integrated set of design interventions and new practices. Arup has used a recent design competition to see what a net zero approach might involve. For solutions to scale and have a tangible effect on cities as a whole, we need to think broadly about how architecture, landscape and public realm design elements can contribute to net zero in an integrated fashion. Our award winning 2021 submission for Hong Kong’s Advancing Net Zero Ideas competition, is a scheme called ‘Taikoo Green Ribbon’ which embodies green design, human experience, ecology and sustainabil

In [91]:
corpus.to_csv('./arup_cities_perspectives_corpus.csv')

In [92]:
corpus.head()

,title,url,date,text
0,"People, nature and technology: an integrated v...",https://www.arup.com/perspectives/people-natur...,NaN,"A net zero built environment, given the sector..."
1,"Planting an idea: Tirana's radical, orbital gr...",https://www.arup.com/perspectives/planting-an-...,NaN,Becoming a more sustainable city means many di...
2,Relighting and rewilding our environment,https://www.arup.com/perspectives/relighting-a...,NaN,In the global sweep of urbanisation over the l...
3,What if the client was everyone? How social va...,https://www.arup.com/perspectives/what-if-the-...,NaN,"As we walk around our cities, we often pass bu..."
4,"People, nature and technology: an integrated v...",https://www.arup.com/perspectives/people-natur...,NaN,"A net zero built environment, given the sector..."
